# Data preparation - ethereum - v1

# Setup

## Library import
We import all the required Python libraries

In [8]:
import os

# Data manipulation
from feature_engine.encoding import RareLabelEncoder, CountFrequencyEncoder
from feature_engine.imputation import MeanMedianImputer
import pandas as pd
import numpy as np

# Visualizations
import plotly
import plotly.graph_objs as go
import plotly.offline as ply
plotly.offline.init_notebook_mode(connected=True)
import matplotlib as plt

import missingno as msno
from pycaret.classification import ClassificationExperiment
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedShuffleSplit
from ydata_profiling import ProfileReport

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
    
%autoreload 2

# Options for pandas
# pd.options.display.max_columns = None
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.float_format', '{:.5f}'.format)
# pd.options.display.float_format = '{:.5f}'.format
# pd.options.display.max_rows = 120

## Local library import
We import all the required local libraries libraries

In [9]:
os.chdir('../')
from src.utils.data_describe import breve_descricao, serie_nulos, cardinalidade, check_for_equal_columns
os.chdir('./notebooks/')

# Parameter definition
We set all relevant parameters for our notebook. By convention, parameters are uppercase, while all the 
other variables follow Python's guidelines.

In [10]:
RAW_FOLDER = '../data/raw/'
INTERIM_FOLDER = '../data/interim/'
PROCESSED_FOLDER = '../data/processed/'
REPORTS_FOLDER = '../reports/'
RANDOM_STATE = 42


# Data import
We retrieve all the required data for the analysis.

In [11]:
try:
    df_raw = pd.read_parquet(INTERIM_FOLDER + 'ethereum_complete.pqt')
    print(f'PARQUET file loaded. Shape: {df_raw.shape}')
except FileNotFoundError as e:
    df_raw = pd.read_csv(RAW_FOLDER + 'ethereum_complete.csv')
    df_raw.drop(columns=['Index'], inplace=True)
   
    df_raw.to_parquet(INTERIM_FOLDER +  'ethereum_complete.pqt', index=False)
    print(f'EXCEL file loaded and PARQUET created. Shape: {df_raw.shape}')

PARQUET file loaded. Shape: (4681, 49)


In [12]:
display(df_raw.head(3))

,Address,FLAG,Avg_min_between_sent_tnx,Avg_min_between_received_tnx,Time_Diff_between_first_and_last_(Mins),Sent_tnx,Received_Tnx,Number_of_Created_Contracts,Unique_Received_From_Addresses,Unique_Sent_To_Addresses,min_value_received,max_value_received,avg_val_received,min_val_sent,max_val_sent,avg_val_sent,min_value_sent_to_contract,max_val_sent_to_contract,avg_value_sent_to_contract,total_transactions_(including_tnx_to_create_contract),total_Ether_sent,total_ether_received,total_ether_sent_contracts,total_ether_balance,Total_ERC20_tnxs,ERC20_total_Ether_received,ERC20_total_ether_sent,ERC20_total_Ether_sent_contract,ERC20_uniq_sent_addr,ERC20_uniq_rec_addr,ERC20_uniq_sent_addr.1,ERC20_uniq_rec_contract_addr,ERC20_avg_time_between_sent_tnx,ERC20_avg_time_between_rec_tnx,ERC20_avg_time_between_rec_2_tnx,ERC20_avg_time_between_contract_tnx,ERC20_min_val_rec,ERC20_max_val_rec,ERC20_avg_val_rec,ERC20_min_val_sent,ERC20_max_val_sent,ERC20_avg_val_sent,ERC20_min_val_sent_contract,ERC20_max_val_sent_contract,ERC20_avg_val_sent_contract,ERC20_uniq_sent_token_name,ERC20_uniq_rec_token_name,ERC20_most_sent_token_type,ERC20_most_rec_token_type
0,0x0020731604c882cf7bf8c444be97d17b19ea4316,1,1457.31000,34.12000,4815.43000,3,13,0,10,3,1.00000,2.50105,1.34844,1.00087,11.27787,5.84292,0,0,0,16,17.52875,17.52978,0,0.00104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
1,0x002bf459dc58584d58886169ea0e80f3ca95ffaf,1,3976.50000,834.77000,9622.53000,2,2,0,1,2,0.58627,0.94751,0.76689,0.58541,0.94728,0.76635,0,0,0,4,1.53269,1.53378,0,0.00109,1.00000,1.33700,0.00000,0.00000,0.00000,1.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,1.33700,1.33700,1.33700,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,,Blockwell say NOTSAFU
2,0x002f0c8119c16d310342d869ca8bf6ace34d9c39,1,112.90000,31.87000,321.42000,2,3,0,3,1,0.00102,0.81780,0.43961,0.50039,0.81751,0.65895,0,0,0,5,1.31790,1.31882,0,0.00092,1.00000,1.33700,0.00000,0.00000,0.00000,1.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,1.33700,1.33700,1.33700,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,,Blockwell say NOTSAFU


# Data cleaning

1. Remove duplicated registries
2. Remove duplicated columns
3. Remove constant columns
4. Remove equal columns
5. Transforming categorical features

## Removing duplicated registries

In [13]:
df_cleaned = df_raw.copy()

In [14]:
df_cleaned.drop_duplicates(inplace=True)

print(f'Shape before data cleasing: {df_raw.shape}')
print(f'After dropping duplicated rows: {df_cleaned.shape}')

Shape before data cleasing: (4681, 49)
After dropping duplicated rows: (4677, 49)


## Removing duplicated columns

In [15]:
lst_duplicated_columns = [
    'ERC20_uniq_sent_addr.1', 'ERC20_avg_time_between_rec_2_tnx', 'ERC20_avg_val_sent_contract',
    'ERC20_max_val_sent_contract', 'ERC20_min_val_sent_contract',
]

df_cleaned.drop(columns=lst_duplicated_columns, inplace=True)

print(f'Shape before data cleasing: {df_raw.shape}')
print(f'After dropping duplicated columns: {df_cleaned.shape}')

Shape before data cleasing: (4681, 49)
After dropping duplicated columns: (4677, 44)


## Removing constant columns

In [16]:
lst_unitary_columns = [
    'total_ether_sent_contracts', 'min_value_sent_to_contract',
    'max_val_sent_to_contract', 'avg_value_sent_to_contract'
]

df_cleaned.drop(columns=lst_unitary_columns, inplace=True)

print(f'Shape before data cleasing: {df_raw.shape}')
print(f'After dropping duplicated columns: {df_cleaned.shape}')

Shape before data cleasing: (4681, 49)
After dropping duplicated columns: (4677, 40)


## Removing equal columns

In [17]:
lst_equal_columns = [
    'min_value_sent_to_contract', 'max_val_sent_to_contract', 'avg_value_sent_to_contract',
    'ERC20_avg_time_between_sent_tnx', 'ERC20_avg_time_between_rec_tnx'
]

for column in lst_equal_columns:
    try:
        df_cleaned.drop(columns=column, inplace=True)
    except:
        print(f"'{column}' was exluded before.")

print(30*'-')
print(f'Shape before data cleasing: {df_raw.shape}')
print(f'After dropping duplicated columns: {df_cleaned.shape}')

'min_value_sent_to_contract' was exluded before.
'max_val_sent_to_contract' was exluded before.
'avg_value_sent_to_contract' was exluded before.
------------------------------
Shape before data cleasing: (4681, 49)
After dropping duplicated columns: (4677, 38)


## Pre-processing categorical features

### Replacing "    " and "0" to "no information"

In [18]:
for column in ['ERC20_most_sent_token_type', 'ERC20_most_rec_token_type']:
    df_cleaned.loc[df_cleaned[column].isnull(), column] = "0"
    for category in [" ", "", "0"]:
        df_cleaned[column] = df_cleaned[column].str.strip()
        df_cleaned[column].replace(category, "no information", inplace=True)
    

    display(
        pd.DataFrame(
            100*df_cleaned.loc[:, column].value_counts()/df_cleaned.loc[:, column].value_counts().sum()
        ).head(10)
    )

,ERC20_most_sent_token_type
no information,89.11696
blockwell.ai KYC Casper Token,1.38978
OmiseGO,0.98354
EOS,0.85525
Golem,0.57729
StatusNetwork,0.47039
Tronix,0.32072
Qtum,0.25657
TenXPay,0.23519
BAT,0.23519


,ERC20_most_rec_token_type
no information,51.40047
Blockwell say NOTSAFU,16.65598
OmiseGO,7.59034
VIU,1.73188
GSENetwork,1.71050
INS Promo,1.15459
Livepeer Token,1.13321
blockwell.ai KYC Casper Token,1.00492
AICRYPTO,0.96216
EOS,0.79111


# Experiment 01 - Only numerical features

In [13]:
df_cleaned_exp_01 = df_cleaned.drop(columns=['ERC20_most_sent_token_type', 'ERC20_most_rec_token_type']).copy()

## Splitting data set

In [14]:
test_size = 0.15
train_size = 1 - test_size

lst_test_sample = df_cleaned_exp_01.sample(int(test_size*len(df_cleaned_exp_01)), random_state=RANDOM_STATE).index.values
lst_train_sample = df_cleaned_exp_01.drop(index=lst_test_sample).index.values

print(f"""
test_size: {test_size}
test_sample (registries): {len(lst_test_sample)}

train_size: {train_size}
train_sample (registries): {len(lst_train_sample)}
""")

df_cleaned_exp_01.loc[lst_train_sample, :].head()


test_size: 0.15
test_sample (registries): 701

train_size: 0.85
train_sample (registries): 3976



,Address,FLAG,Avg_min_between_sent_tnx,Avg_min_between_received_tnx,Time_Diff_between_first_and_last_(Mins),Sent_tnx,Received_Tnx,Number_of_Created_Contracts,Unique_Received_From_Addresses,Unique_Sent_To_Addresses,min_value_received,max_value_received,avg_val_received,min_val_sent,max_val_sent,avg_val_sent,total_transactions_(including_tnx_to_create_contract),total_Ether_sent,total_ether_received,total_ether_balance,Total_ERC20_tnxs,ERC20_total_Ether_received,ERC20_total_ether_sent,ERC20_total_Ether_sent_contract,ERC20_uniq_sent_addr,ERC20_uniq_rec_addr,ERC20_uniq_rec_contract_addr,ERC20_avg_time_between_contract_tnx,ERC20_min_val_rec,ERC20_max_val_rec,ERC20_avg_val_rec,ERC20_min_val_sent,ERC20_max_val_sent,ERC20_avg_val_sent,ERC20_uniq_sent_token_name,ERC20_uniq_rec_token_name
0,0x0020731604c882cf7bf8c444be97d17b19ea4316,1,1457.31000,34.12000,4815.43000,3,13,0,10,3,1.00000,2.50105,1.34844,1.00087,11.27787,5.84292,16,17.52875,17.52978,0.00104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0x002bf459dc58584d58886169ea0e80f3ca95ffaf,1,3976.50000,834.77000,9622.53000,2,2,0,1,2,0.58627,0.94751,0.76689,0.58541,0.94728,0.76635,4,1.53269,1.53378,0.00109,1.00000,1.33700,0.00000,0.00000,0.00000,1.00000,1.00000,0.00000,1.33700,1.33700,1.33700,0.00000,0.00000,0.00000,0.00000,1.00000
2,0x002f0c8119c16d310342d869ca8bf6ace34d9c39,1,112.90000,31.87000,321.42000,2,3,0,3,1,0.00102,0.81780,0.43961,0.50039,0.81751,0.65895,5,1.31790,1.31882,0.00092,1.00000,1.33700,0.00000,0.00000,0.00000,1.00000,1.00000,0.00000,1.33700,1.33700,1.33700,0.00000,0.00000,0.00000,0.00000,1.00000
3,0x0059b14e35dab1b4eee1e2926c7a5660da66f747,1,2300.37000,65.10000,73091.00000,29,98,0,89,26,0.00078,15.72907,0.38322,0.00000,36.70000,1.31496,127,38.13377,37.55605,-0.57772,96.00000,142677.38290,120354.76840,0.00000,6.00000,55.00000,37.00000,0.00000,0.00000,26436.08100,1954.48470,0.00000,81324.07460,5232.81602,22.00000,37.00000
4,0x005b9f4516f8e640bbe48136901738b323c53b00,1,0.00000,0.00000,0.00000,0,0,0,0,0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0,0.00000,0.00000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Exporting

In [15]:
df_cleaned_exp_01.loc[lst_train_sample, :].to_parquet(INTERIM_FOLDER + 'df_train_exp_01.pqt')
df_cleaned_exp_01.loc[lst_test_sample, :].to_parquet(INTERIM_FOLDER + 'df_test_exp_01.pqt')

# Experiment 02 - Categorical features encoded by frequency

In [19]:
df_cleaned_exp_02 = df_cleaned.copy()

## Splitting data set

In [50]:
test_size = 0.15
train_size = 1 - test_size

X = df_cleaned_exp_02.drop(columns=['Address', 'FLAG']).copy()
y = df_cleaned_exp_02[['FLAG']]

df_cleaned_train_exp_02, df_cleaned_test_exp_02 = train_test_split(
    df_cleaned_exp_02.drop(columns=['Address']), train_size=train_size, 
    random_state=RANDOM_STATE
)

print(f"""
test_size: {test_size}
test_sample (registries): {len(df_cleaned_test_exp_02)}

train_size: {train_size}
train_sample (registries): {len(df_cleaned_train_exp_02)}
""")

df_cleaned_train_exp_02.head()


test_size: 0.15
test_sample (registries): 702

train_size: 0.85
train_sample (registries): 3975



,FLAG,Avg_min_between_sent_tnx,Avg_min_between_received_tnx,Time_Diff_between_first_and_last_(Mins),Sent_tnx,Received_Tnx,Number_of_Created_Contracts,Unique_Received_From_Addresses,Unique_Sent_To_Addresses,min_value_received,max_value_received,avg_val_received,min_val_sent,max_val_sent,avg_val_sent,total_transactions_(including_tnx_to_create_contract),total_Ether_sent,total_ether_received,total_ether_balance,Total_ERC20_tnxs,ERC20_total_Ether_received,ERC20_total_ether_sent,ERC20_total_Ether_sent_contract,ERC20_uniq_sent_addr,ERC20_uniq_rec_addr,ERC20_uniq_rec_contract_addr,ERC20_avg_time_between_contract_tnx,ERC20_min_val_rec,ERC20_max_val_rec,ERC20_avg_val_rec,ERC20_min_val_sent,ERC20_max_val_sent,ERC20_avg_val_sent,ERC20_uniq_sent_token_name,ERC20_uniq_rec_token_name,ERC20_most_sent_token_type,ERC20_most_rec_token_type
3691,0,67691.12000,0.00000,135382.23000,2,1,0,1,2,0.70927,0.70927,0.70927,0.10700,0.29554,0.20127,3,0.40254,0.70927,0.30673,7.00000,139.77633,0.00000,0.00000,0.00000,5.00000,7.00000,0.00000,0.00000,77.37208,19.96805,0.00000,0.00000,0.00000,0.00000,7.00000,no information,VIU
152,1,23488.73000,33.36000,48178.53000,2,36,0,26,2,0.01038,9.99000,1.33005,0.07579,48.30435,24.19007,38,48.38014,47.88187,-0.49828,2.00000,1.33700,0.00000,0.00000,0.00000,2.00000,2.00000,0.00000,0.00000,1.33700,0.66850,0.00000,0.00000,0.00000,0.00000,2.00000,no information,Blockwell say NOTSAFU
1175,1,0.00000,0.00000,0.00000,0,0,0,0,0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0,0.00000,0.00000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no information,no information
3873,0,0.00000,1420.72000,198901.02000,0,140,1,3,0,0.00000,5.04889,2.10414,0.00000,0.00000,0.00000,141,0.00000,294.57965,294.57965,2.00000,4.90150,0.00000,0.00000,0.00000,2.00000,2.00000,0.00000,0.20608,4.69542,2.45075,0.00000,0.00000,0.00000,0.00000,2.00000,no information,OmiseGO
3011,0,22.18000,0.00000,66.55000,3,1,0,1,3,101.00000,101.00000,101.00000,0.61000,93.11405,33.66615,4,100.99845,101.00000,0.00155,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,no information,no information


## Preprocessing

In [51]:

lst_numerical_columns = df_cleaned_train_exp_02.drop(columns=[
    'FLAG', 'ERC20_most_sent_token_type', 'ERC20_most_rec_token_type'
]).columns.tolist()

pipe = Pipeline([
    ('rare', RareLabelEncoder(
        n_categories=2, max_n_categories=3, replace_with='rare', tol=0.10,
        variables=['ERC20_most_sent_token_type', 'ERC20_most_rec_token_type']
    )),
    ('frq', CountFrequencyEncoder(
        encoding_method='frequency',
        variables=['ERC20_most_sent_token_type', 'ERC20_most_rec_token_type']
    )),
    ('imputer', MeanMedianImputer(
        imputation_method='median',
        variables=lst_numerical_columns
    ))
])

df_cleaned_train_exp_02_transformed = pipe.fit_transform(df_cleaned_train_exp_02)
df_cleaned_test_exp_02_transformed = pipe.transform(df_cleaned_test_exp_02)

display(df_cleaned_train_exp_02_transformed['ERC20_most_rec_token_type'].value_counts())
display(df_cleaned_train_exp_02_transformed['ERC20_most_sent_token_type'].value_counts())

0.51421    2044
0.31874    1267
0.16704     664
Name: ERC20_most_rec_token_type, dtype: int64

0.89006    3538
0.10994     437
Name: ERC20_most_sent_token_type, dtype: int64

### Effect of transformation onver numerical features

In [54]:
df_num_features_before_transf = df_cleaned_train_exp_02.loc[:, lst_numerical_columns].describe().T[['count', 'mean', 'std', '50%']]
df_num_features_after_transf = df_cleaned_train_exp_02_transformed.loc[:, lst_numerical_columns].describe().T[['count', 'mean', 'std', '50%']]

pd.concat([df_num_features_before_transf, df_num_features_after_transf], axis=1).to_excel(PROCESSED_FOLDER + 'df_after_preprocessing.xlsx')

In [33]:
lst_numeric_columns = df_cleaned_exp_02.select_dtypes(exclude=object).columns.tolist()
lst_numeric_columns.remove('FLAG')
print(len(lst_numeric_columns), lst_numeric_columns)

34 ['Avg_min_between_sent_tnx', 'Avg_min_between_received_tnx', 'Time_Diff_between_first_and_last_(Mins)', 'Sent_tnx', 'Received_Tnx', 'Number_of_Created_Contracts', 'Unique_Received_From_Addresses', 'Unique_Sent_To_Addresses', 'min_value_received', 'max_value_received', 'avg_val_received', 'min_val_sent', 'max_val_sent', 'avg_val_sent', 'total_transactions_(including_tnx_to_create_contract)', 'total_Ether_sent', 'total_ether_received', 'total_ether_balance', 'Total_ERC20_tnxs', 'ERC20_total_Ether_received', 'ERC20_total_ether_sent', 'ERC20_total_Ether_sent_contract', 'ERC20_uniq_sent_addr', 'ERC20_uniq_rec_addr', 'ERC20_uniq_rec_contract_addr', 'ERC20_avg_time_between_contract_tnx', 'ERC20_min_val_rec', 'ERC20_max_val_rec', 'ERC20_avg_val_rec', 'ERC20_min_val_sent', 'ERC20_max_val_sent', 'ERC20_avg_val_sent', 'ERC20_uniq_sent_token_name', 'ERC20_uniq_rec_token_name']


In [35]:
pd.DataFrame(
    df_cleaned_train_exp_02_transformed.loc[:, lst_numeric_columns].median(axis=0), columns=['Mediana']
).to_excel(PROCESSED_FOLDER + 'preprocessing_numerical.xlsx')

## Exporting

In [37]:
df_cleaned_train_exp_02.to_parquet(INTERIM_FOLDER + 'df_train_exp_02.pqt')
df_cleaned_test_exp_02.to_parquet(INTERIM_FOLDER + 'df_test_exp_02.pqt')